In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\kannapr1\Downloads\Fraudulent_Claims_Dataset.csv")

In [3]:
df.head()

,Claim_ID,Claim_Date,Policy_Issuance_Date,Customer_ID,Claim_Amount,Claim_Type,Annual_Income,Suspicious_Flags,Fraud_Label
0,CLM000001,10-10-2020,21-12-2012,CUST4020,20635.68,Home,120769.55,0,0
1,CLM000002,03-06-2023,02-06-2018,CUST8985,34662.22,Life,412022.47,0,0
2,CLM000003,06-04-2023,05-01-2022,CUST2792,64847.97,Home,110227.23,0,0
3,CLM000004,10-03-2023,29-04-2014,CUST9410,16906.56,Life,91553.79,0,0
4,CLM000005,26-01-2023,08-09-2018,CUST1702,64860.47,Life,53075.22,1,1


In [4]:
df['Claim-to-Income Ratio'] =  df['Claim_Amount'] / df['Annual_Income']

In [5]:
df[['Claim-to-Income Ratio', 'Claim_Amount', 'Annual_Income']].head()

,Claim-to-Income Ratio,Claim_Amount,Annual_Income
0,0.170868,20635.68,120769.55
1,0.084127,34662.22,412022.47
2,0.588312,64847.97,110227.23
3,0.184663,16906.56,91553.79
4,1.222048,64860.47,53075.22


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Claim_ID               5000 non-null   object 
 1   Claim_Date             5000 non-null   object 
 2   Policy_Issuance_Date   5000 non-null   object 
 3   Customer_ID            5000 non-null   object 
 4   Claim_Amount           5000 non-null   float64
 5   Claim_Type             5000 non-null   object 
 6   Annual_Income          5000 non-null   float64
 7   Suspicious_Flags       5000 non-null   int64  
 8   Fraud_Label            5000 non-null   int64  
 9   Claim-to-Income Ratio  5000 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 390.8+ KB


In [7]:
df['Policy_Issuance_Date'] = pd.to_datetime(df['Policy_Issuance_Date'], format="%d-%m-%Y")

In [8]:
df["Claim_Date"] = pd.to_datetime(df["Claim_Date"], format="%d-%m-%Y")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Claim_ID               5000 non-null   object        
 1   Claim_Date             5000 non-null   datetime64[ns]
 2   Policy_Issuance_Date   5000 non-null   datetime64[ns]
 3   Customer_ID            5000 non-null   object        
 4   Claim_Amount           5000 non-null   float64       
 5   Claim_Type             5000 non-null   object        
 6   Annual_Income          5000 non-null   float64       
 7   Suspicious_Flags       5000 non-null   int64         
 8   Fraud_Label            5000 non-null   int64         
 9   Claim-to-Income Ratio  5000 non-null   float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(3)
memory usage: 390.8+ KB


In [10]:
df['FiledAfterPolicy'] = (df['Claim_Date'] -df['Policy_Issuance_Date']).dt.days

In [11]:
df["Filed_Early"] = df["FiledAfterPolicy"].apply(lambda x: 1 if x <= 100 else 0)

In [12]:
df["Filed_Early"].unique()

array([0, 1], dtype=int64)

In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Claim_Type_encoded'] = encoder.fit_transform(df['Claim_Type'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Claim_ID               5000 non-null   object        
 1   Claim_Date             5000 non-null   datetime64[ns]
 2   Policy_Issuance_Date   5000 non-null   datetime64[ns]
 3   Customer_ID            5000 non-null   object        
 4   Claim_Amount           5000 non-null   float64       
 5   Claim_Type             5000 non-null   object        
 6   Annual_Income          5000 non-null   float64       
 7   Suspicious_Flags       5000 non-null   int64         
 8   Fraud_Label            5000 non-null   int64         
 9   Claim-to-Income Ratio  5000 non-null   float64       
 10  FiledAfterPolicy       5000 non-null   int64         
 11  Filed_Early            5000 non-null   int64         
 12  Claim_Type_encoded     5000 non-null   int32         
dtypes: 

In [15]:
from sklearn.ensemble import IsolationForest

In [16]:
isoforest = IsolationForest(contamination = 0.05, random_state=42)

In [17]:
df['ISO_Outliers'] = isoforest.fit_predict(df[['Claim_Amount', 'Claim-to-Income Ratio', 'Filed_Early', 'Claim_Type_encoded']])

In [18]:
df['ISO_Outliers'].unique()

array([ 1, -1])

In [19]:
df['ISO_Outliers'] = df['ISO_Outliers'].apply(lambda x:1 if x==-1 else 0)

In [20]:
df['ISO_Outliers'].value_counts()

ISO_Outliers
0    4750
1     250
Name: count, dtype: int64

In [21]:
df['Suspicious_Flags']  = df['ISO_Outliers'] 

In [22]:
df['Suspicious_Flags'].value_counts()


Suspicious_Flags
0    4750
1     250
Name: count, dtype: int64

In [23]:
df['Fraud_Label'].value_counts()

Fraud_Label
0    4256
1     744
Name: count, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Claim_ID               5000 non-null   object        
 1   Claim_Date             5000 non-null   datetime64[ns]
 2   Policy_Issuance_Date   5000 non-null   datetime64[ns]
 3   Customer_ID            5000 non-null   object        
 4   Claim_Amount           5000 non-null   float64       
 5   Claim_Type             5000 non-null   object        
 6   Annual_Income          5000 non-null   float64       
 7   Suspicious_Flags       5000 non-null   int64         
 8   Fraud_Label            5000 non-null   int64         
 9   Claim-to-Income Ratio  5000 non-null   float64       
 10  FiledAfterPolicy       5000 non-null   int64         
 11  Filed_Early            5000 non-null   int64         
 12  Claim_Type_encoded     5000 non-null   int32         
 13  ISO

In [25]:
df[df['Suspicious_Flags']==1]

,Claim_ID,Claim_Date,Policy_Issuance_Date,Customer_ID,Claim_Amount,Claim_Type,Annual_Income,Suspicious_Flags,Fraud_Label,Claim-to-Income Ratio,FiledAfterPolicy,Filed_Early,Claim_Type_encoded,ISO_Outliers
72,CLM000073,2021-07-30,2019-08-01,CUST6501,18275.92,Auto,2462.94,1,0,7.420368,729,0,0,1
83,CLM000084,2022-10-24,2018-12-29,CUST3726,22502.73,Auto,10797.22,1,1,2.084123,1395,0,0,1
99,CLM000100,2024-12-09,2021-06-04,CUST6299,79893.86,Life,24814.45,1,0,3.219651,1284,0,2,1
122,CLM000123,2022-05-13,2020-08-26,CUST3125,95418.74,Life,52911.87,1,0,1.803352,625,0,2,1
128,CLM000129,2024-02-24,2017-08-17,CUST8956,19754.47,Auto,7773.95,1,0,2.541111,2382,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898,CLM004899,2023-07-04,2015-03-05,CUST5995,68457.54,Home,31063.46,1,1,2.203796,3043,0,1,1
4900,CLM004901,2021-08-09,2018-08-04,CUST3741,8440.33,Medical,6354.59,1,0,1.328226,1101,0,3,1
4978,CLM004979,2023-08-04,2021-05-11,CUST2345,70996.29,Life,2283.39,1,1,31.092494,815,0,2,1
4979,CLM004980,2021-08-24,2020-06-18,CUST6259,17171.05,Home,6613.66,1,1,2.596301,432,0,1,1


In [26]:
df = df.drop(columns=['FiledAfterPolicy','ISO_Outliers'])

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score

In [57]:
df.head()

,Claim_ID,Claim_Date,Policy_Issuance_Date,Customer_ID,Claim_Amount,Claim_Type,Annual_Income,Suspicious_Flags,Fraud_Label,Claim-to-Income Ratio,Filed_Early,Claim_Type_encoded
0,CLM000001,2020-10-10,2012-12-21,CUST4020,20635.68,Home,120769.55,0,0,0.170868,0,1
1,CLM000002,2023-06-03,2018-06-02,CUST8985,34662.22,Life,412022.47,0,0,0.084127,0,2
2,CLM000003,2023-04-06,2022-01-05,CUST2792,64847.97,Home,110227.23,0,0,0.588312,0,1
3,CLM000004,2023-03-10,2014-04-29,CUST9410,16906.56,Life,91553.79,0,0,0.184663,0,2
4,CLM000005,2023-01-26,2018-09-08,CUST1702,64860.47,Life,53075.22,0,1,1.222048,0,2


In [55]:
X = df[['Filed_Early', 'Claim-to-Income Ratio', 'Claim_Amount', 'Claim_Type_encoded']]

In [57]:
Y = df['Fraud_Label']

In [59]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, random_state=42, test_size=0.2)

In [61]:
scaler  = StandardScaler()

In [63]:
X_Train_scaled = scaler.fit_transform(X_Train)
X_Test_scaled = scaler.transform(X_Test)

In [65]:
rfmodel = RandomForestClassifier(random_state=42, n_estimators=100)

In [67]:
rfmodel.fit(X_Train, Y_Train)

RandomForestClassifier(random_state=42)

In [69]:
Y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5000 entries, 0 to 4999
Series name: Fraud_Label
Non-Null Count  Dtype
--------------  -----
5000 non-null   int64
dtypes: int64(1)
memory usage: 39.2 KB


In [71]:
Y.value_counts()

Fraud_Label
0    4256
1     744
Name: count, dtype: int64

In [73]:
df['Suspicious_Flags'].value_counts()

Suspicious_Flags
0    4750
1     250
Name: count, dtype: int64

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Claim_ID               5000 non-null   object        
 1   Claim_Date             5000 non-null   datetime64[ns]
 2   Policy_Issuance_Date   5000 non-null   datetime64[ns]
 3   Customer_ID            5000 non-null   object        
 4   Claim_Amount           5000 non-null   float64       
 5   Claim_Type             5000 non-null   object        
 6   Annual_Income          5000 non-null   float64       
 7   Suspicious_Flags       5000 non-null   int64         
 8   Fraud_Label            5000 non-null   int64         
 9   Claim-to-Income Ratio  5000 non-null   float64       
 10  Filed_Early            5000 non-null   int64         
 11  Claim_Type_encoded     5000 non-null   int32         
dtypes: datetime64[ns](2), float64(3), int32(1), int64(3), object(3

In [77]:
nn_model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=500, random_state=42)

In [79]:
nn_model.fit(X_Train_scaled, Y_Train)

MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)

In [80]:
rf_probability_train = rfmodel.predict_proba(X_Train)[:,1]
rf_probability_test = rfmodel.predict_proba(X_Test)[:,1]

In [81]:
rf_probability_test

array([0.02, 0.02, 0.74, 0.  , 0.  , 0.  , 0.  , 0.  , 0.37, 0.01, 0.04,
       0.  , 0.  , 0.63, 0.04, 0.77, 0.94, 0.  , 0.7 , 0.01, 0.02, 0.91,
       0.01, 0.02, 0.99, 0.  , 0.27, 0.01, 0.12, 0.35, 0.  , 0.02, 0.  ,
       0.  , 0.02, 0.  , 0.  , 0.01, 0.01, 0.06, 0.03, 0.08, 0.62, 0.  ,
       0.12, 0.08, 0.  , 0.2 , 0.66, 0.02, 0.06, 0.  , 0.  , 0.  , 0.01,
       0.71, 0.02, 0.52, 0.07, 0.08, 0.02, 0.  , 0.01, 0.  , 0.06, 0.04,
       0.12, 0.  , 0.47, 0.11, 0.18, 0.07, 0.  , 0.  , 0.  , 0.01, 0.01,
       0.  , 0.25, 0.03, 0.65, 0.01, 0.  , 0.  , 0.72, 0.62, 0.  , 0.76,
       0.  , 0.78, 0.01, 0.04, 0.02, 0.  , 0.74, 0.25, 0.06, 0.  , 0.  ,
       0.74, 0.  , 0.02, 0.71, 0.42, 0.84, 0.01, 0.01, 0.01, 0.06, 0.79,
       0.  , 0.06, 0.46, 0.03, 0.37, 0.  , 0.01, 0.02, 0.  , 0.  , 0.  ,
       0.03, 0.  , 0.  , 0.06, 0.05, 0.  , 0.8 , 0.7 , 0.01, 0.  , 0.75,
       0.  , 0.7 , 0.08, 0.02, 0.  , 0.  , 0.23, 0.78, 0.02, 0.08, 0.1 ,
       0.12, 0.8 , 0.02, 0.  , 0.  , 0.38, 0.07, 0.

In [82]:
nn_probability_train = nn_model.predict_proba(X_Train_scaled)[:,1]
nn_probability_test = nn_model.predict_proba(X_Test_scaled)[:,1]

In [83]:
nn_probability_test

array([0.02071944, 0.03761323, 0.52469967, 0.05323765, 0.03189875,
       0.04875984, 0.0362147 , 0.02741004, 0.58196805, 0.03098221,
       0.02168769, 0.03111557, 0.03712571, 0.55833289, 0.02670737,
       0.30866473, 0.4458392 , 0.04532783, 0.3416068 , 0.0291568 ,
       0.0494829 , 0.74815201, 0.02265114, 0.03380596, 0.73653565,
       0.03121886, 0.38531012, 0.02383051, 0.02584174, 0.57782557,
       0.02459774, 0.01946004, 0.02236756, 0.02236906, 0.02062746,
       0.08714278, 0.04905826, 0.04065941, 0.0233478 , 0.0336326 ,
       0.02530937, 0.02160669, 0.79830644, 0.01897801, 0.02381492,
       0.12026125, 0.04732084, 0.03164996, 0.44281417, 0.02641062,
       0.17299573, 0.02965446, 0.03900299, 0.03091808, 0.18700963,
       0.62191953, 0.02288889, 0.60771639, 0.03217442, 0.05131374,
       0.02059435, 0.02187902, 0.02381902, 0.0254861 , 0.04095915,
       0.03079799, 0.05327351, 0.03527294, 0.0500664 , 0.07096654,
       0.05322588, 0.06195598, 0.03988575, 0.02124222, 0.02212

In [89]:
fraud_score_train = (rf_probability_train + nn_probability_train)/2

In [91]:
fraud_score_test = (rf_probability_test + nn_probability_test)/2

In [93]:
train_results = pd.DataFrame({'Fraud_Score': fraud_score_train, 'Actual_Label': Y_Train.values})
test_results = pd.DataFrame({'Fraud_Score': fraud_score_test, 'Actual_Label': Y_Test.values})

In [95]:
train_results.head()

,Fraud_Score,Actual_Label
0,0.020174,0
1,0.431896,0
2,0.012818,0
3,0.498989,0
4,0.011553,0


In [99]:
df.loc[X_Train.index, 'Fraud_Score'] = fraud_score_train
df.loc[X_Test.index, 'Fraud_Score'] = fraud_score_test

In [101]:
df

,Claim_ID,Claim_Date,Policy_Issuance_Date,Customer_ID,Claim_Amount,Claim_Type,Annual_Income,Suspicious_Flags,Fraud_Label,Claim-to-Income Ratio,Filed_Early,Claim_Type_encoded,Fraud_Score
0,CLM000001,2020-10-10,2012-12-21,CUST4020,20635.68,Home,120769.55,0,0,0.170868,0,1,0.020160
1,CLM000002,2023-06-03,2018-06-02,CUST8985,34662.22,Life,412022.47,0,0,0.084127,0,2,0.035269
2,CLM000003,2023-04-06,2022-01-05,CUST2792,64847.97,Home,110227.23,0,0,0.588312,0,1,0.025610
3,CLM000004,2023-03-10,2014-04-29,CUST9410,16906.56,Life,91553.79,0,0,0.184663,0,2,0.034976
4,CLM000005,2023-01-26,2018-09-08,CUST1702,64860.47,Life,53075.22,0,1,1.222048,0,2,0.741450
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CLM004996,2023-01-04,2020-03-18,CUST3914,35815.93,Medical,446398.45,0,0,0.080233,0,3,0.026871
4996,CLM004997,2024-11-19,2017-07-06,CUST3568,88092.00,Life,46434.22,0,1,1.897135,0,2,0.503645
4997,CLM004998,2022-09-12,2019-11-16,CUST9544,48806.95,Medical,106002.92,0,0,0.460430,0,3,0.029293
4998,CLM004999,2021-02-05,2013-05-16,CUST8225,95711.64,Life,332103.76,0,0,0.288198,0,2,0.011320


In [103]:
df['Fraud_Prediction'] = (df['Fraud_Score'] >= 0.75).astype(int)

In [105]:
df

,Claim_ID,Claim_Date,Policy_Issuance_Date,Customer_ID,Claim_Amount,Claim_Type,Annual_Income,Suspicious_Flags,Fraud_Label,Claim-to-Income Ratio,Filed_Early,Claim_Type_encoded,Fraud_Score,Fraud_Prediction
0,CLM000001,2020-10-10,2012-12-21,CUST4020,20635.68,Home,120769.55,0,0,0.170868,0,1,0.020160,0
1,CLM000002,2023-06-03,2018-06-02,CUST8985,34662.22,Life,412022.47,0,0,0.084127,0,2,0.035269,0
2,CLM000003,2023-04-06,2022-01-05,CUST2792,64847.97,Home,110227.23,0,0,0.588312,0,1,0.025610,0
3,CLM000004,2023-03-10,2014-04-29,CUST9410,16906.56,Life,91553.79,0,0,0.184663,0,2,0.034976,0
4,CLM000005,2023-01-26,2018-09-08,CUST1702,64860.47,Life,53075.22,0,1,1.222048,0,2,0.741450,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CLM004996,2023-01-04,2020-03-18,CUST3914,35815.93,Medical,446398.45,0,0,0.080233,0,3,0.026871,0
4996,CLM004997,2024-11-19,2017-07-06,CUST3568,88092.00,Life,46434.22,0,1,1.897135,0,2,0.503645,0
4997,CLM004998,2022-09-12,2019-11-16,CUST9544,48806.95,Medical,106002.92,0,0,0.460430,0,3,0.029293,0
4998,CLM004999,2021-02-05,2013-05-16,CUST8225,95711.64,Life,332103.76,0,0,0.288198,0,2,0.011320,0
